In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
#from osgeo import gdal

%cd windpath
%cd 12_sample


D:\dvc\windpath
D:\dvc\windpath\12_sample


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten

time_steps = 9
width = 1500
height = 1200
channels = 7

model = Sequential([
    keras.Input(shape=(None,channels,height,width)),
    layers.ConvLSTM2D(filters=300, kernel_size=(3, 3), padding="same", return_sequences=True),
    layers.BatchNormalization(),
    layers.ConvLSTM2D(filters=100, kernel_size=(3, 3), padding="same", return_sequences=True),
    layers.BatchNormalization(),
    layers.ConvLSTM2D(filters=50, kernel_size=(3, 3), padding="same", return_sequences=True),
    layers.BatchNormalization(),
    layers.Conv3D(filters=1,kernel_size=(3,3,3),activation="sigmoid",padding="same"),
])

# 모델 컴파일
model.compile(optimizer='adadelta', loss='mse', metrics=['r2'])
model.summary()

In [ ]:
X = []
Y = []
path_prefix = "klam21_"
for i in range(1, 2):
  for h in range(1, 3):
    for s in range(0,6):
      for d in range(0, 360, 60):
        height = np.full((1200,1500),h)
        speed = np.full((1200,1500), s)
        direction = np.full((1200,1500),d)
        path = path_prefix + "0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)
        landuse = np.loadtxt(path +'/landuse.txt', delimiter = ' ', skiprows = 6, dtype = 'int')
        terrain = np.loadtxt(path +'/terrain.txt', delimiter = ' ', skiprows = 6, dtype = 'int')
        for j in range(360, 3960, 360):
            uz = np.loadtxt(path+'/result/'+"0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_uz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            vz = np.loadtxt(path+'/result/'+"0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_vz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            stacked_layer = np.dstack((uz,vz,landuse,terrain,direction,speed,height))
            if j == 3600:
                Y.append(stacked_layer)
            else :
                X.append(stacked_layer)

In [29]:
X = np.array(X)
Y = np.array(Y)

MemoryError: Unable to allocate 33.8 GiB for an array with shape (720, 1200, 1500, 7) and data type int32

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(x=X, y=Y, validation_split=0.2, epochs=50, callbacks=[early_stop])